In [3]:
from shared.library import *
from shared.extract_profile import *

# Read the data from the Excel file
excel_file = 'grid_data_sheet.xlsx'

# Step 1: Create the network with generation
net = create_network(excel_file)

This pandapower network includes the following parameter tables:
   - bus (11 element)
   - load (3 element)
   - sgen (7 element)
   - ext_grid (2 element)
   - line (14 element)
   - bus_geodata (11 element)


# please refer the simbench database for profiles #
https://github.com/e2nIEE/simbench/blob/master/tutorials/simbench_grids_basics_and_usage.ipynb

In [4]:
import simbench as sb
import pandas as pd
import numpy as np
import pandapower.timeseries as ts
from pandapower.control.controller.const_control import ConstControl
from pandapower.timeseries.data_sources.frame_data import DFData


In [5]:
samples_per_month = 31 * 24 * 60 // 15

In [6]:
"""load the load p,q profile from simbench dataset"""
grid_code = "1-HV-mixed--0-sw"
profiles = sb.get_absolute_values(sb.get_simbench_net(grid_code), profiles_instead_of_study_cases=True)
load_p = profiles[("load", "p_mw")]
load_q = profiles[("load", "q_mvar")]
load_proportion_1 = 2 / 5
load_proportion_2 = 2 / 5
load_proportion_3 = 1 / 5
# Extract the first month's data
one_month_data = load_p.iloc[:samples_per_month].sum(axis=1) # Sum the groups with their respective proportions
load_1_p = one_month_data * load_proportion_1
load_2_p = one_month_data * load_proportion_2
load_3_p = one_month_data * load_proportion_3
load_p_sum = pd.DataFrame({
    'load_1': load_1_p,
    'load_2': load_2_p,
    'load_3': load_3_p
})

one_month_data = load_q.iloc[:samples_per_month].sum(axis=1) # Sum the groups with their respective proportions
load_1_q = one_month_data * load_proportion_1
load_2_q = one_month_data * load_proportion_2
load_3_q = one_month_data * load_proportion_3
load_q_sum= pd.DataFrame({
    'load_1': load_1_q,
    'load_2': load_2_q,
    'load_3': load_3_q
})

"""create datasource and initialize controller"""
ds_load = DFData(load_p_sum)
ConstControl(net, element='load', element_index=net.load.index[0], variable='p_mw', data_source=ds_load,
             profile_name='load_1')
ConstControl(net, element='load', element_index=net.load.index[1], variable='p_mw', data_source=ds_load,
             profile_name='load_2')
ConstControl(net, element='load', element_index=net.load.index[2], variable='p_mw', data_source=ds_load,
             profile_name='load_3')
ds_load = DFData(load_q_sum)
ConstControl(net, element='load', element_index=net.load.index[0], variable='q_mvar', data_source=ds_load,
             profile_name='load_1')
ConstControl(net, element='load', element_index=net.load.index[1], variable='q_mvar', data_source=ds_load,
             profile_name='load_2')
ConstControl(net, element='load', element_index=net.load.index[2], variable='q_mvar', data_source=ds_load,
             profile_name='load_3')

This ConstControl has the following parameters: 

index:             5
json_excludes:     ['self', '__class__']

In [7]:
"""add some noise to the wind generation data"""
df = pd.DataFrame(np.random.normal(1., 0.1, size=(samples_per_month, len(net.sgen.index))),
              index=list(range(samples_per_month)), columns=net.sgen.index) * net.sgen.p_mw.values

"""create datasource from it"""
ds_sgen = DFData(df)

"""initialising ConstControl controller to update values"""
ConstControl(net, element='sgen', element_index=net.sgen.index,variable='p_mw', data_source=ds_sgen, profile_name=net.sgen.index)

This ConstControl has the following parameters: 

index:             6
json_excludes:     ['self', '__class__']

In [8]:
ow = ts.OutputWriter(net, output_path="./", output_file_type=".xlsx")
ow.log_variable("res_load", "p_mw")
ow.log_variable("res_load", "q_mvar")
ow.log_variable("res_sgen", "p_mw")
ts.run_timeseries(net)

100%|██████████| 2976/2976 [00:16<00:00, 175.59it/s]


In [11]:
print(load_p_sum.shape)
print(load_q_sum.shape)
print(df.shape)  # For sgen
print(len(range(0,samples_per_month)))

(2976, 3)
(2976, 3)
(2976, 7)
2976


In [14]:
import numpy as np
import pandas as pd

import pandapower.control as control
import pandapower.networks as nw
import pandapower.timeseries as timeseries
from pandapower.timeseries.data_sources.frame_data import DFData

# load a pandapower network
net = nw.mv_oberrhein(scenario='generation')
# number of time steps
n_ts = 95
# load your timeseries from a file (here csv file)
# df = pd.read_csv("sgen_timeseries.csv")
# or create a DataFrame with some random time series as an example
df = pd.DataFrame(np.random.normal(1., 0.1, size=(n_ts, len(net.sgen.index))),
                  index=list(range(n_ts)), columns=net.sgen.index) * net.sgen.p_mw.values
print(df.shape)
# create the data source from it
ds = DFData(df)

# initialising ConstControl controller to update values of the regenerative generators ("sgen" elements)
# the element_index specifies which elements to update (here all sgens in the net since net.sgen.index is passed)
# the controlled variable is "p_mw"
# the profile_name are the columns in the csv file (here this is also equal to the sgen indices 0-N )
const_sgen = control.ConstControl(net, element='sgen', element_index=net.sgen.index,
                                  variable='p_mw', data_source=ds, profile_name=net.sgen.index)

# do the same for loads
# df = pd.read_csv("load_timeseries.csv")
# create a DataFrame with some random time series as an example
df = pd.DataFrame(np.random.normal(1., 0.1, size=(n_ts, len(net.load.index))),
                  index=list(range(n_ts)), columns=net.load.index) * net.load.p_mw.values
print(df.shape)
ds = DFData(df)
const_load = control.ConstControl(net, element='load', element_index=net.load.index,
                                  variable='p_mw', data_source=ds, profile_name=net.load.index)

# initialising the outputwriter to save data to excel files in the current folder. You can change this to .json, .csv, or .pickle as well
ow = timeseries.OutputWriter(net, output_path="./", output_file_type=".xlsx")
# adding vm_pu of all buses and line_loading in percent of all lines as outputs to be stored
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')

# starting the timeseries simulation for one day -> 96 15 min values.
timeseries.run_timeseries(net)

(95, 153)
(95, 147)


100%|██████████| 95/95 [00:00<00:00, 111.89it/s]


In [15]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (179 element)
   - load (147 element)
   - sgen (153 element)
   - switch (322 element)
   - ext_grid (2 element)
   - line (181 element)
   - trafo (2 element)
   - controller (2 element)
   - line_geodata (181 element)
   - bus_geodata (179 element)
   - output_writer (1 elements)
 and the following results tables:
   - res_bus (179 element)
   - res_line (181 element)
   - res_trafo (2 element)
   - res_ext_grid (2 element)
   - res_load (147 element)
   - res_sgen (153 element)
   - res_switch (322 element)


In [16]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,LV Load 0,103,0.223763,0.050765,0.0,0.0,NaN,0.1,True,MV/LV Station
1,LV Load 1,174,0.646362,0.127927,0.0,0.0,NaN,0.1,True,MV/LV Station
2,LV Load 2,194,0.300515,0.050765,0.0,0.0,NaN,0.1,True,MV/LV Station
3,LV Load 3,34,0.284162,0.050765,0.0,0.0,NaN,0.1,True,MV/LV Station
4,LV Load 4,76,0.232049,0.050765,0.0,0.0,NaN,0.1,True,MV/LV Station
5,LV Load 5,159,0.399284,0.081223,0.0,0.0,NaN,0.1,True,MV/LV Station
6,LV Load 6,38,0.287090,0.050765,0.0,0.0,NaN,0.1,True,MV/LV Station
7,LV Load 7,157,0.356868,0.081223,0.0,0.0,NaN,0.1,True,MV/LV Station
8,LV Load 8,168,0.647801,0.127927,0.0,0.0,NaN,0.1,True,MV/LV Station
9,LV Load 9,216,0.440104,0.081223,0.0,0.0,NaN,0.1,True,MV/LV Station
